### Cointegration Pair Trading

Based on this [NEDL video](https://www.youtube.com/watch?v=jvZ0vuC9oJk&t=5s)

In [4]:
# Import packages
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import scipy.optimize as spop
import matplotlib.pyplot as plt

print("Yahoo Finance: ", yf.__version__)

Yahoo Finance:  0.2.52


In [ ]:
# Specifying parameters
stocks = ['JPM', 'C']
start = '2019-12-31'
end = '2021-03-08'
fee = 0.001
window = 252
# Threshold value of the t-statistics from unit root regression.
# The more negative the t-statistics, the better.
# Rules of thumbs: should be less than -2.5.
# -3 is very good, -4 is excellent, -5 is wonderful.
t_threshold = -2.5

# Retrieving data
data = pd.DataFrame()
for stock in stocks:
    prices = yf.download(stock, start, end)
    data[stock] = prices['Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [20]:
# Calculate returns
returns = pd.DataFrame()
for stock in stocks:
    df_t = data[stock][1:].reset_index(drop=True)
    df_tm1 = data[stock][:-1].reset_index(drop=True)
    r = df_t / df_tm1 - 1.0
    returns[stock] = np.append(r, 0)

print(returns)

          JPM         C
0    0.012124  0.016773
1   -0.013196 -0.018836
2   -0.000795 -0.003137
3   -0.017001 -0.008685
4    0.007801  0.007618
..        ...       ...
292 -0.003256 -0.009060
293  0.019332  0.021332
294 -0.015369 -0.008099
295  0.002324  0.006017
296  0.000000  0.000000

[297 rows x 2 columns]


In [ ]:
# Initializing arrays
gross_returns = np.array([])
net_returns = np.array([])
t_s = np.array([])
stock1 = stocks[0]
stock2 = stocks[1]
for t in range(window, len(data)):
    # Define the unit root function: stock2 = a + b * stock1